In [2]:
import pandas as pd
import json
import re
from string import punctuation
from konlpy.tag import Mecab
import datetime
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
# mecab = Mecab()

In [3]:
TIME_MAPPING = {}
TIME_MAPPING[0] = (datetime.datetime(1, 1, 1, 16, 00), datetime.datetime(1, 1, 2, 8, 59)) # 하루 전 조건 넣어줘야함
i = 1
START = datetime.datetime(1,1,2,9,0)

s = START
while s.strftime('%H:%M') < "16:00":
    e = s + datetime.timedelta(minutes = 4)
    TIME_MAPPING[i] = (s, e)

    s += datetime.timedelta(minutes=5)
    i += 1
TIME_MAPPING[len(TIME_MAPPING)] = (s, )

In [ ]:
# len(TIME_MAPPING)
# TIME_MAPPING[86]
TIME_MAPPING
# s

# pororo 라벨링, 1차 단어사전 만들기

In [ ]:
%%writefile naver_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('Naver종토방')[1:]:
  NAVER = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('Naver종토방/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa),
      NAVER = NAVER.append(j, ignore_index = True)
      NAVER.to_pickle(f"./naver_all_{f}.pkl")

  # pd.read_pickle('./naver_all.pkl').append(pd.read_pickle('./naver_all2.pkl'), ignore_index= True).to_pickle('./naver_all.pkl')

In [ ]:
%%writefile dc_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('DC갤러리'):
  DC = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('DC갤러리/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa)
      DC = DC.append(j, ignore_index = True)
      DC.to_pickle(f"./dc_all_{f}.pkl")

In [ ]:
# 백그라운드로 파일 실행
# !nohup /usr/bin/python3 naver_pickle.py > naver.log 2>&1 &
# !nohup /usr/bin/python3 dc_pickle.py > dc.log 2>&1 &

# log 확인
# !tail -f naver.log
# !tail -f dc.log

# 감성사전 만들기

mecab 체ㄴ, 용ㅓ, 감탄ㅏ(확ㅣ), (관형ㅏ)
{토큰 : 감ㅓ확률, 토ㄴ: 감ㅓ확률}


In [ ]:
mecab.tagset

In [54]:
def data_preprocessing(pkl : str):
  df = pd.read_pickle(pkl)
  types = {i : 'int16' for i in df.columns if i not in ['_id', 'code', 'date', 'title', 'nsmc', 'shop']}
  df = df.astype(dtype = types)
  df.date = pd.to_datetime(df.date)
  # 중복 제거
  df.drop_duplicates(subset = ['code', 'title'], inplace=True, ignore_index= True)
  df['positive'] = (df.nsmc + df.shop) / 2
  # 전처리
  # Stopwords 처리
  pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
  pattern2 = re.compile(r'[^가-힣! ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
  pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

  df.title = df.title.map(lambda x: pattern3.sub(' ',
                                                    pattern2.sub('',
                                                                    pattern1.sub('', x))))
  global mecab
  mecab = Mecab()
  # mecab 토큰화
  # df['mecab'] = df.title.apply(lambda x: mecab.morphs(x))
  df['mecab'] = df.title.apply(lambda x: mecab.pos(x))
  # N(체언), V(용언), IC(감탄사)만 거르기
  for m in df.mecab.values:
    for w, t in m:
        # print(m)
        if t.startswith('N') or t.startswith("V") or t == 'IC': pass
        else: m.remove((w, t))
  
  return df

In [ ]:
NAVER = data_preprocessing('./naver_all.pkl').rename(columns = {'views':'view'})
DC = data_preprocessing('./dc_all.pkl')

In [ ]:
# naver = NAVER.drop(columns = ['_id', 'nsmc', 'shop'])
# DATA = NAVER[['code', 'date', 'mecab', 'view', 'positive']].append(DC[['code', 'date', 'mecab', 'view', 'positive']],
                                                                    # ignore_index = True)
DATA = NAVER.drop(columns=['_id', 'pos', 'neg']).append(DC.drop(columns = ['recommend', '_id']), ignore_index = True)
DATA[['code', 'date', 'title', 'mecab', 'view', 'positive']]

In [ ]:
# DATA.to_pickle("./DATA.pkl")
DATA = pd.read_pickle("./DATA.pkl")

In [ ]:
SENTIMENT = {}

In [ ]:
for data in DATA[['mecab', 'view', 'positive']].values:
    # print(data[0])
    for w, t in data[0]: # mecab tokens
        if w in SENTIMENT.keys():
            SENTIMENT[w].append((data[1], data[2]))
        else:
            SENTIMENT[w] = [(data[1], data[2])]
    # break

In [ ]:
ex = pd.DataFrame(SENTIMENT['바닥'], columns = ['view', 'positive'])
ex['token'] = '바닥'
ex = ex[['token', 'positive', 'view']]
ex

In [ ]:
# ex['weight'] = ex.view / ex.view.sum()
ex['positive'] = ex.positive * ex.weight
ex.groupby('token').sum().reset_index()

In [ ]:
for key, values in SENTIMENT.items():
    one_view = 0
    one_positive = 0
    for v in values: one_view += v[0]
    for v in values: one_positive += (v[0] / one_view) * v[1]

    SENTIMENT[key] = one_positive

token별 조회수 가중합  
  
    
$\frac{v_1}{v_1 + ... + v_n} * positive_1 + \frac{v_2}{v_1 + ... + v_n} * positive_2 + ... + \frac{v_n}{v_1 + ... + v_n} * positive_n = POSITIVE_{token} $

In [ ]:
# pd.DataFrame.from_dict(SENTIMENT, orient= 'index').reset_index().rename(columns = {'index': 'token', 0:'positive'})

SENTIMENT['바닥']

In [ ]:
# json.dump(SENTIMENT, open('./SENTIMENT_ver1.0.json', 'w'), ensure_ascii=False)
pickle.dump(SENTIMENT, open('SENTIMENT_ver1.0.pkl', 'wb'))

In [ ]:
# json.load(open('./SENTIMENT_ver1.0.json', 'r'))
SENTIMENT = pickle.load(open('SENTIMENT_ver1.0.pkl', 'rb'))
# SENTIMENT

# STOCK_PRICE 5분단위 & time_mapping

In [ ]:
import sqlite3
conn = sqlite3.connect('/Users/yejin/Desktop/Career/Projects/AYOA/Stock.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM stock")
STOCK = pd.DataFrame(columns = ['pk', 'code', 'date', 'time', 'price'], data = cursor.fetchall()).drop(columns = 'pk')

In [ ]:
STOCK.head(10)

In [ ]:
STOCK_PRICE = pd.DataFrame(columns = ['code', 'date', 'time', 'price'])

for i in tqdm(range(len(STOCK))):
    if int(STOCK.time[i][-2:]) % 5 == 0:
        STOCK_PRICE = STOCK_PRICE.append(STOCK.iloc[i], ignore_index=True)
del STOCK

In [ ]:
STOCK_PRICE

In [ ]:
STOCK_PRICE.to_pickle('./STOCK.pkl')

In [ ]:
STOCK_PRICE = pd.read_pickle('./STOCK.pkl')
STOCK_PRICE.date = STOCK_PRICE.date + " " + STOCK_PRICE.time
STOCK_PRICE.date = STOCK_PRICE.date.astype('datetime64')
STOCK_PRICE.price = STOCK_PRICE.price.str.replace(',', "").astype('int')

for i in tqdm(range(len(STOCK_PRICE))):
    for k in range(86):
        if STOCK_PRICE.date[i].time() == TIME_MAPPING[k][0].time():
            STOCK_PRICE.loc[i, 'time_map'] = k
            break
STOCK_PRICE.to_pickle('./STOCK_PRICE_map.pkl')

In [ ]:
del STOCK

# mecab_데이터 time_mapping

In [ ]:
DATA = pd.read_pickle('./DATA.pkl')
# DATA

In [ ]:
DATA.head()

In [ ]:
DATA['time_map'] = 0 # 기본 0
TIME_MAPPING[85] = (datetime.datetime(1, 1, 1, 16, 0), ) # for loop

for i in tqdm(range(len(DATA))):
    for k in range(1,85):
        if DATA.date[i].time() >= TIME_MAPPING[k][0].time() and DATA.date[i].time() < TIME_MAPPING[k+1][0].time():
            DATA.loc[i, 'time_map'] = k
            break
        # else: DATA.loc[i, 'time_map'] = 0

In [ ]:
DATA.to_pickle("./DATA_map.pkl")

In [ ]:
# DATA.time_map.astype('int16')
DATA = pd.read_pickle('./DATA_map.pkl')
DATA.info()

In [ ]:
# NEW_DATA = DATA[DATA.time_map != 0]
# NEW_DATA = DATA[DATA.time_map.notnull()].reset_index(drop = True)
# 9시에 8:55~8:59 데이터 가져오니까
# NEW_DATA.time_map += 1
del DATA

# 주가 등락 적용 업데이트
- 10:03 게시글 -> 긍정확률에 수식 적용하려면 % 올랐는지가 있어야 하지 -> 10시 5분에 비해 10:10분 기반으로 올랐는지 내렸는지 학습
- 18:50 게시글 -> 9:00에 비해 9:05가 올랐는지, 09:05 기반으로 학습
- pdf -> 하루종일 영향을 준다. (안 올라오는 경우가 더 많음. 올라올 때만 추가적인 정보로) +a


### 등락 수식 (감성사전 업데이트 식)
- n% 변화, $x_t = t시점의 단어 감성점수$
- 시간별로 업데이트
- known에 대해서 점수 업데이트 > 문장의 감성점수를 구할 수 있음(평균) > unknown initialize
    - known 부정/긍정 :  $x_{t+1} = x_t + n * x_t * max(해당 분위 단어들의 감성점수)$ > $min(max(x_{t+1}, 0), 1)$ > 짜잔

ex. 삼성에 대해 100개 리뷰(9시 0분 ~ 9시 4분)  
-> 100개 리뷰 전체 word counting > 30% 30% 40% 나누고 > 해당 분위 감성점수를 더 곱해줌(위 수식)  
-> a 50번 b 2번의 경우

---------
### 종목별 확률 (inference 식) ; 예측할 때
- 리뷰의 감성점수 도출: 토큰화된 리뷰의 감성 점수 평균 = 한 리뷰의 감성 점수
- 가중치 :  
- pdf 점수랑 합치는 수식 : 알파 = 0.3 예상

## 기간 맞추기

In [4]:
OPINION = pd.read_pickle("./DATA_map.pkl")
STOCK_PRICE = pd.read_pickle('./STOCK_PRICE_map.pkl')

In [5]:
STOCK_PRICE['date_time'] = STOCK_PRICE.date
STOCK_PRICE['time'] = STOCK_PRICE.date.apply(lambda x: x.time())
STOCK_PRICE.date = STOCK_PRICE.date.apply(lambda x: x.date()) 

OPINION['date_time'] = OPINION.date
OPINION['time'] = OPINION.date.apply(lambda x: x.time())
OPINION['date'] = OPINION.date.apply(lambda x: x.date())

In [ ]:
DATA = OPINION.drop(columns = ['title', 'nsmc', 'shop', 'time'])
DATA.time_map += 1
# 10시 3분 게시글? 10시 5분 주가에 영향
# 장 마감 이후 게시글? 14시 이후는 다음날 9시, 08시 이전은 당일 9시
for i in tqdm(range(len(DATA))):
    if DATA.date_time[i].time() >= datetime.time(16, 0):
        DATA.loc[i, 'price_time'] = datetime.datetime.combine((DATA.date_time[i] + datetime.timedelta(days = 1)).date(),datetime.time(9,0))
    elif DATA.date_time[i].time() < datetime.time(9, 0):
       DATA.loc[i, 'price_time'] = datetime.datetime.combine(DATA.date_time[i].date(), datetime.time(9,0))
    else:
        DATA.loc[i, 'price_time'] = datetime.datetime.combine(DATA.date_time[i].date(), TIME_MAPPING[DATA.time_map[i]][0].time())
DATA

In [ ]:
STOCK_PRICE

In [ ]:
DATA = DATA.drop(columns = ['date', 'time_map']).merge(STOCK_PRICE.drop(columns = ['date', 'time_map', 'time']).rename(columns = {"date_time": "price_time"}), on = ['code', 'price_time'])

# time_pred = 예측할 가격 시간 (15:10), price_pred = 예측할 가격 >  change : price & price_prd pct_change()
DATA['time_pred'] = DATA.price_time + datetime.timedelta(minutes = 5)
DATA = DATA.merge(STOCK_PRICE[['code', 'date_time', 'price']].rename(columns = {"date_time": 'time_pred'}), 
        on = ['code', 'time_pred']).rename(columns = {'price_x' : 'price', 'price_y': 'price_pred'})

DATA

In [ ]:
DATA['change'] = (DATA['price_pred'] - DATA.price) / DATA.price_pred #* 100
DATA

In [ ]:
data = DATA.copy()
data.mecab = data.mecab.astype("str")
dup_idx = data[data.duplicated()].index

In [ ]:
DATA = DATA.drop(index = dup_idx).reset_index(drop = True)

In [ ]:
DATA

In [ ]:
DATA.to_pickle("./DATA_change.pkl")

## 등락수식 적용

In [7]:
DATA = pd.read_pickle('./DATA_change.pkl')
DATA.change = DATA.change / 100

In [8]:
# 종목별 분위 나누기
TICKER = DATA.code.unique()
# TICKER

In [9]:
from collections import Counter
# wc를 map-reduce로? 과한듯
for i in tqdm(range(len(DATA))):
    DATA.mecab[i] = [(w, DATA.change[i]) for w, _ in DATA.mecab[i]]

100%|██████████| 86487/86487 [00:54<00:00, 1573.49it/s]


In [10]:
ST = pickle.load(open('SENTIMENT_ver1.0.pkl', 'rb'))
SENTIMENT = ST.copy()

In [11]:
# 5분 단위 투자자 반응
for i in tqdm(range(len(DATA.price_time.unique()))):
    one_term = DATA[DATA.price_time == DATA.price_time.unique()[i]]
    for t in TICKER:
    # t = TICKER[0]
        tokens = one_term[one_term.code == t].mecab.sum()
        # print(t, tokens)
        if (tokens == 0) or (len(tokens) == 0): continue

        word_count = Counter([i[0] for i in tokens]).most_common()
        # 분위 나누기
        r1 = [i[0] for i in word_count[:int(len(word_count) * .3)]]
        r2 = [i[0] for i in word_count[int(len(word_count) * .3):int(len(word_count) * .6)]]
        r3 = [i[0] for i in word_count[int(len(word_count) * .6):]]
        if len(r1) == 0: max_r1 = 0
        else: max_r1 = max([SENTIMENT[i] for i in r1])
        # print('max_r1 : ', max_r1)
        if len(r2) == 0: max_r2 = 0
        else: max_r2 = max([SENTIMENT[i] for i in r2])
        if len(r3) == 0: max_r1 = 0
        else: max_r3 = max([SENTIMENT[i] for i in r3])
        # print('max_r : ', (max_r1, max_r2, max_r3))

        # 단어의 여러 change 값 평균 내기
        token_df = pd.DataFrame(tokens, columns = ['token', 'change'])
        token_df = token_df.groupby('token').mean()['change']

        # 감성사전 업데이트
        for one in r1: SENTIMENT[one] = min(max(SENTIMENT[one] * (1 + max_r1 * token_df[one]), 0), 1)
        for one in r2: SENTIMENT[one] = min(max(SENTIMENT[one] * (1 + max_r2 * token_df[one]), 0), 1)
        for one in r3: SENTIMENT[one] = min(max(SENTIMENT[one] * (1 + max_r3 * token_df[one]), 0), 1)

100%|██████████| 3339/3339 [00:48<00:00, 68.37it/s] 


In [ ]:
SENTIMENT
# word_count
# token_df

In [42]:
# 예시 캡쳐파일
one_term = DATA[DATA.price_time == DATA.price_time.unique()[30]]
tokens = one_term[one_term.code == '035720'].mecab.sum()
        # print(t, tokens)
# if (tokens == 0) or (len(tokens) == 0): continue

word_count = Counter([i[0] for i in tokens]).most_common()
# 분위 나누기
r1 = [i[0] for i in word_count[:int(len(word_count) * .3)]]
r2 = [i[0] for i in word_count[int(len(word_count) * .3):int(len(word_count) * .6)]]
r3 = [i[0] for i in word_count[int(len(word_count) * .6):]]

In [49]:
# one_term[one_term.code == '035720']
ex = pd.DataFrame(word_count, columns = ['token', 'word_count'])
ex['rank'] = [1 for _ in range(int(len(word_count) * .3))] + [2 for _ in range(int(len(word_count) * .3),int(len(word_count) * .6))]\
            + [3 for _ in range(int(int(len(word_count) * .4)))]
ex

,token,word_count,rank
0,카카오,5,1
1,있,2,1
2,나,2,1
3,평단,2,1
4,높,2,1
5,사람,2,1
6,거머리,1,1
7,기생충,1,1
8,망해라,1,1
9,정부,1,2


In [13]:
pickle.dump(SENTIMENT, open('SENTIMENT_ver2.0.pkl', 'wb'))

## 새로운 리뷰로 리뷰 감성점수 업데이트

In [84]:
NAVER = pd.DataFrame()
for line in open('../NAVER_0120.json', 'r', encoding = 'utf8').readlines():
    j = json.loads(line)
    NAVER = NAVER.append(j, ignore_index = True)
NAVER = NAVER.drop(columns = ["_id", 'pos', 'neg']).rename(columns = {'views':'view'})
NAVER

,code,date,title,view
0,008770,2022.01.20 10:55,나는 호신을 믿습니다,34
1,008770,2022.01.20 10:32,이제는 7.5도 힘들겠다.,58
2,008770,2022.01.20 10:12,리포트 실적 때문에 이런건가요?,103
3,008770,2022.01.20 09:48,답답하다,201
4,008770,2022.01.20 09:36,아 진짜 드럽게 안오르네,171
...,...,...,...,...
2104,030350,2022.01.20 09:04,다른 NFT 회사들 다오르는데 애 왜이래 ㅎㅎ;;,251
2105,030350,2022.01.20 08:47,결과물이 점점 나오기 시작 리전오브 소식도좀 올려줘,132
2106,030350,2022.01.20 08:31,페루 공략,202
2107,030350,2022.01.20 08:00,오늘,136


In [85]:
DC = pd.DataFrame()
for line in open('../DC_0120.json', 'r', encoding = 'utf8').readlines():
    j = json.loads(line)
    DC = DC.append(j, ignore_index = True)
DC = DC.drop(columns = ["_id", 'recommend'])
DC

,code,date,title,view
0,035720,2022-01-20 11:21:31,카카오 조수용·여민수 공동대표 사퇴,24
1,035720,2022-01-20 11:16:56,카카오 사라 기회다 ㅇㅇㅇㅇㅇㅇㅇㅇ나는 말햇다,44
2,035720,2022-01-20 10:21:59,카카오 주주 어떻게하냐...,139
3,035720,2022-01-20 10:17:58,카카오 좌빨이라 주가회복 없는거아니냐?,71
4,035720,2022-01-20 10:05:11,셀트리온 오스템임플란트 삼성전자 카카오 한국비엔씨,42
...,...,...,...,...
58,005930,2022-01-19 17:45:24,혼수 자격?,26
59,005930,2022-01-19 12:43:04,능력있는 놈이 다 못생긴건 아닌데,64
60,005930,2022-01-19 11:43:58,우리나라 주식은 삼성전자 밖에 없음,98
61,005930,2022-01-19 11:42:50,삼성전자도 반도체만 빼서 물적분할 상장해라 500조 갈 듯ㅋㅋ,55


In [86]:
def preprocessing(df):
    # df.date = df.date.astype('datetime64')
    pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
    pattern2 = re.compile(r'[^가-힣! ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
    pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

    df.title = df.title.map(lambda x: pattern3.sub(' ',
                                                    pattern2.sub('',
                                                                    pattern1.sub('', x))))
    global mecab
    mecab = Mecab()
    # mecab 토큰화
    # df['mecab'] = df.title.apply(lambda x: mecab.morphs(x))
    df['mecab'] = df.title.apply(lambda x: mecab.pos(x))
    # N(체언), V(용언), IC(감탄사)만 거르기
    for m in df.mecab.values:
        for w, t in m:
            # print(m)
            if t.startswith('N') or t.startswith("V") or t == 'IC': pass
            else: m.remove((w, t))
    return df[['code', 'date', 'title', 'mecab', 'view']]

In [87]:
NAVER = preprocessing(NAVER)
DC = preprocessing(DC)

In [88]:
DATA = NAVER.append(DC, ignore_index = True)
DATA

,code,date,title,mecab,view
0,008770,2022.01.20 10:55,나는 호신을 믿습니다,"[(나, NP), (호신, NNG), (믿, VV)]",34
1,008770,2022.01.20 10:32,이제는 도 힘들겠다,"[(이제, NNG), (도, NNG), (힘들, VA), (다, EC)]",58
2,008770,2022.01.20 10:12,리포트 실적 때문에 이런건가요,"[(리포트, NNG), (실적, NNG), (때문, NNB), (이런, MM), (...",103
3,008770,2022.01.20 09:48,답답하다,"[(하, XSA)]",201
4,008770,2022.01.20 09:36,아 진짜 드럽게 안오르네,"[(아, IC), (드럽, VA), (안, MAG), (오르, VV)]",171
...,...,...,...,...,...
2167,005930,2022-01-19 17:45:24,혼수 자격,"[(혼수, NNG), (자격, NNG)]",26
2168,005930,2022-01-19 12:43:04,능력있는 놈이 다 못생긴건 아닌데,"[(능력, NNG), (있, VV), (놈, NNB), (다, MAG), (못생긴,...",64
2169,005930,2022-01-19 11:43:58,우리나라 주식은 삼성전자 밖에 없음,"[(우리, NP), (나라, NNG), (주식, NNG), (삼성전자, NNP), ...",98
2170,005930,2022-01-19 11:42:50,삼성전자도 반도체만 빼서 물적분할 상장해라 조 갈 듯,"[(삼성전자, NNP), (반도체, NNG), (빼, VV), (물적분할, NNP)...",55


In [89]:
data = DATA.copy()
for i in range(len(DATA)):
    data.loc[i, 'mecab'] = [(w, SENTIMENT[w]) if w in SENTIMENT.keys() else (w, -1) for w, _ in data.loc[i, 'mecab']]

In [90]:
data.date = data.date.astype('datetime64')

In [91]:
data[data.code == '035720']
# set(TICKER) - set(data.code.unique())

,code,date,title,mecab,view
116,035720,2022-01-20 11:13:00,쓰레기 정경유착 기업,"[(쓰레기, 0.03240633628453415), (정경, 0.2601309040...",4
117,035720,2022-01-20 11:12:00,좌파정권에서 해처먹을때 좋았지,"[(좌파, 0.18327308181098395), (정권, 0.32815619630...",5
118,035720,2022-01-20 11:12:00,내일부터 반등 크게 나온단다 돈잘 버네,"[(내일, 0.35589813779610885), (반, 0.361969278268...",8
119,035720,2022-01-20 11:12:00,껍데기 카카오,"[(껍데기, 0.04532279932299497), (카카오, 0.237465863...",10
120,035720,2022-01-20 11:11:00,블록딜 끝 지금 팔지마시고 무조건,"[(블록, 0.27172246487070684), (딜, 0.251558271125...",47
...,...,...,...,...,...
2152,035720,2022-01-19 09:01:01,정권바뀌면 제일 불안한게 카카오아니냐,"[(정권, 0.3281561963069665), (바뀌, 0.318709218355...",190
2153,035720,2022-01-19 08:59:32,카카오는 진짜 역대급 양아치 기업인듯,"[(카카오, 0.23746586374235407), (진짜, 0.2452949706...",234
2154,035720,2022-01-19 08:58:18,카카오 진짜 개같이 멸망했네,"[(카카오, 0.23746586374235407), (개, 0.18217070969...",117
2155,035720,2022-01-19 08:50:25,카카오 풀매수 해도됨,"[(카카오, 0.23746586374235407), (풀, 0.47276975674...",409


In [101]:
DATA = data
DATA['time_map'] = 0
for i in tqdm(range(len(DATA))):
    for k in range(1,85):
        if DATA.date[i].time() >= TIME_MAPPING[k][0].time() and DATA.date[i].time() < TIME_MAPPING[k+1][0].time():
            DATA.loc[i, 'time_map'] = k
            break

100%|██████████| 2172/2172 [00:01<00:00, 2059.69it/s]


In [103]:
DATA[DATA.code == '035720']

,code,date,title,mecab,view,time_map
116,035720,2022-01-20 11:13:00,쓰레기 정경유착 기업,"[(쓰레기, 0.03240633628453415), (정경, 0.2601309040...",4,27
117,035720,2022-01-20 11:12:00,좌파정권에서 해처먹을때 좋았지,"[(좌파, 0.18327308181098395), (정권, 0.32815619630...",5,27
118,035720,2022-01-20 11:12:00,내일부터 반등 크게 나온단다 돈잘 버네,"[(내일, 0.35589813779610885), (반, 0.361969278268...",8,27
119,035720,2022-01-20 11:12:00,껍데기 카카오,"[(껍데기, 0.04532279932299497), (카카오, 0.237465863...",10,27
120,035720,2022-01-20 11:11:00,블록딜 끝 지금 팔지마시고 무조건,"[(블록, 0.27172246487070684), (딜, 0.251558271125...",47,27
...,...,...,...,...,...,...
2152,035720,2022-01-19 09:01:01,정권바뀌면 제일 불안한게 카카오아니냐,"[(정권, 0.3281561963069665), (바뀌, 0.318709218355...",190,1
2153,035720,2022-01-19 08:59:32,카카오는 진짜 역대급 양아치 기업인듯,"[(카카오, 0.23746586374235407), (진짜, 0.2452949706...",234,0
2154,035720,2022-01-19 08:58:18,카카오 진짜 개같이 멸망했네,"[(카카오, 0.23746586374235407), (개, 0.18217070969...",117,0
2155,035720,2022-01-19 08:50:25,카카오 풀매수 해도됨,"[(카카오, 0.23746586374235407), (풀, 0.47276975674...",409,0


In [128]:
# # -1 인 감성점수 업데이트
data = DATA.copy()
# data.date = data.date.astype('str')
data['positive'] = -1

for i in range(len(DATA)):
    sent = 0
    ct = 0
    for w, p in data.loc[i, 'mecab']:
        if p != -1 : 
            sent += p
            ct += 1
    if ct != 0:
        data.loc[i, 'positive'] = sent / ct
        # data.loc[i, 'mecab'] = [(x, y) if y != -1 else (x, sent / ct) for x, y in data.loc[i, 'mecab']]

In [129]:
for i in range(len(data)):
    data.mecab[i] = [(x, y) if y != -1 else (x, sent / ct) for x, y in data.loc[i, 'mecab']]
    for w, p in data.loc[i, 'mecab']: SENTIMENT[w] = p

In [131]:
pickle.dump(SENTIMENT, open('SENTIMENT_ver2.0119.pkl', 'wb'))

In [132]:
DATA = data
DATA

,code,date,title,mecab,view,time_map,positive
0,008770,2022-01-20 10:55:00,나는 호신을 믿습니다,"[(나, 0.2790970843277334), (호신, 0.2811130705077...",34,24,0.320514
1,008770,2022-01-20 10:32:00,이제는 도 힘들겠다,"[(이제, 0.2582961856666353), (도, 0.2621155185015...",58,19,0.243312
2,008770,2022-01-20 10:12:00,리포트 실적 때문에 이런건가요,"[(리포트, 0.33178038459644954), (실적, 0.3610796098...",103,15,0.235418
3,008770,2022-01-20 09:48:00,답답하다,"[(하, 0.19541603661871795)]",201,10,0.195416
4,008770,2022-01-20 09:36:00,아 진짜 드럽게 안오르네,"[(아, 0.2720924013950172), (드럽, 0.1416124545389...",171,8,0.222513
...,...,...,...,...,...,...,...
2167,005930,2022-01-19 17:45:24,혼수 자격,"[(혼수, 0.2173375577799614), (자격, 0.205113397841...",26,0,0.211225
2168,005930,2022-01-19 12:43:04,능력있는 놈이 다 못생긴건 아닌데,"[(능력, 0.12295170713171584), (있, 0.302598866432...",64,45,0.191433
2169,005930,2022-01-19 11:43:58,우리나라 주식은 삼성전자 밖에 없음,"[(우리, 0.4032008229635947), (나라, 0.288111200725...",98,33,0.305113
2170,005930,2022-01-19 11:42:50,삼성전자도 반도체만 빼서 물적분할 상장해라 조 갈 듯,"[(삼성전자, 0.3584341241929131), (반도체, 0.354845678...",55,33,0.299379


In [144]:
DATA.view = DATA.view.astype('int')
DATA[(DATA.code == '035720') & (DATA.date.astype('str').str.startswith('2022-01-19')) & (DATA.time_map == 4)]

,code,date,title,mecab,view,time_map,positive
2148,035720,2022-01-19 09:19:02,카카오는 윤석열 되면 개같이 멸망할듯,"[(카카오, 0.23746586374235407), (윤석열, 0.274815488...",201,4,0.230787
2149,035720,2022-01-19 09:17:52,카카오 설거지빔,"[(카카오, 0.23746586374235407), (설거지, 0.167072099...",100,4,0.166251
2150,035720,2022-01-19 09:16:04,카카오 착한 척 오졌던 거 같은데,"[(카카오, 0.23746586374235407), (착한, 0.4950597159...",138,4,0.275566


In [149]:
df = DATA[(DATA.code == '035720') & (DATA.date.astype('str').str.startswith('2022-01-19')) & (DATA.time_map == 4)]
one_total_view = df.view.sum()
one_positive = 0
# for v, _ in df[['view', 'positive']].values: one_view += v

for v, p in df[['view', 'positive']].values: one_positive += (v / one_total_view) * p 

one_positive

# sent = 

0.23016281216652584

In [150]:
TIME_MAPPING[4]

(datetime.datetime(1, 1, 2, 9, 15), datetime.datetime(1, 1, 2, 9, 19))

In [212]:
# 5분 단위 종목별 평균 등락
UPDOWN = pd.DataFrame(columns= ['code', 'time_'time_pred', 'sentiment', 'title'])
d = "2022-01-20"
for t in TICKER:
    DF = DATA[(DATA.code == t) & (DATA.date.astype('str').str.startswith(d))]
    for tm in DF.time_map.unique():
        df = DF[DF.time_map == tm].reset_index(drop = True)
        if len(df) == 0 : continue
        one_total_view = df.view.sum()
        one_positive = 0
        for v, p in df[['view', 'positive']].values: one_positive += (v / one_total_view) * p
        UPDOWN = UPDOWN.append({'code': t, 'time_pred': datetime.datetime.combine(df.date[0].date(), TIME_MAPPING[tm + 2][0].time()),
                                'sentiment': one_positive, 'title': df.title.sum()}, ignore_index = True)


In [213]:
# DATA[(DATA.code == '035720') & (DATA.time_map == 4) & (DATA.date.astype('str').str.startswith('2022-01-19'))]

In [214]:
# TIME_MAPPING[5]

In [209]:
UPDOWN.loc[[18]]

,code,time_pred,sentiment,title
18,035720,2022-01-19 09:25:00,0.230163,카카오는 윤석열 되면 개같이 멸망할듯 카카오 설거지빔 카카오 착한 척 오졌던 거 같은데


## 정확도 확인

In [219]:
STOCK_PRICE = json.load(open('../stock_0120.json', 'r'))

In [225]:
# DATA[DATA.date.astype('str').str.startswith('2022-01-19')]
UPDOWN[UPDOWN.time_pred.astype('str').str.startswith('2022-01-19')]

,code,time_pred,sentiment,title
0,035720,2022-01-19 09:05:00,0.276548,이재명 카카오페이 먹튀 처벌근데 카카오페이 임원들 주식판거 후폭풍 생각안했을까씹좌파...
1,035720,2022-01-19 14:40:00,0.340303,최근 시장에서 트리플 크라운 달성한 사람도 있겠지
2,035720,2022-01-19 14:35:00,0.252118,카톡말고 다른거 쓸만한거 없냐
3,035720,2022-01-19 13:25:00,0.187767,경찰 카카오 김범수 의장 탈세 의혹 고발 사건 수사 착수
4,035720,2022-01-19 13:20:00,0.177910,손절한 주식이 꼬라박으면 그것도 참 즐거워
5,035720,2022-01-19 13:05:00,0.263493,카카오 비중 인데 물타기 지금할까
6,035720,2022-01-19 13:00:00,0.219286,카카오 물린 개미들은 봐라카카오 시바새키보이스피싱 전화왔는데 이거 위험하노
7,035720,2022-01-19 12:50:00,0.211349,카카오 진짜 하 씨발카카오 주가 개노답이네
8,035720,2022-01-19 12:20:00,0.130612,씨발 국장 망했냐
9,035720,2022-01-19 12:15:00,0.288407,카카오뱅크 만원대 물렸는데 상반기 회복될까요


In [227]:
STOCK_PRICE['035720']

[['20220120', '15:58', '92,300'],
 ['20220120', '15:57', '92,300'],
 ['20220120', '15:56', '92,300'],
 ['20220120', '15:55', '92,300'],
 ['20220120', '15:54', '92,300'],
 ['20220120', '15:53', '92,300'],
 ['20220120', '15:52', '92,300'],
 ['20220120', '15:51', '92,300'],
 ['20220120', '15:50', '92,300'],
 ['20220120', '15:49', '92,300'],
 ['20220120', '15:48', '92,300'],
 ['20220120', '15:47', '92,300'],
 ['20220120', '15:46', '92,300'],
 ['20220120', '15:45', '92,300'],
 ['20220120', '15:44', '92,300'],
 ['20220120', '15:43', '92,300'],
 ['20220120', '15:42', '92,300'],
 ['20220120', '15:41', '92,300'],
 ['20220120', '15:40', '92,300'],
 ['20220120', '15:30', '92,300'],
 ['20220120', '15:19', '92,000'],
 ['20220120', '15:18', '92,000'],
 ['20220120', '15:17', '92,000'],
 ['20220120', '15:16', '92,100'],
 ['20220120', '15:15', '92,200'],
 ['20220120', '15:14', '92,200'],
 ['20220120', '15:13', '92,100'],
 ['20220120', '15:12', '92,200'],
 ['20220120', '15:11', '92,100'],
 ['20220120', 